# Niave Bayes Classification
1. Clean data
    1. Compute average of regular season for each team each season
    1. Seperate post season data
    1. Make sure final out come is present

In [1]:
import pandas as pd
# !pip install google-cloud-bigquery
from google.cloud import bigquery
from google.oauth2 import service_account
# !pip install db_dtypes 
import db_dtypes 

In [2]:
project_id = 'dataanalyticscs5850'
credentials = service_account.Credentials.from_service_account_file('serviceUserKey.json')

client = bigquery.Client(credentials= credentials, project= project_id)

In [3]:
dataset_ref = client.dataset('ncaa_basketball', project='bigquery-public-data')
ncaa_dataset = client.get_dataset(dataset_ref)
# gameData = client.get_table(ncaa_dataset.table('mbb_games_sr'))
# print(type(gameData))
# gameData = pd.DataFrame(gameData)

query="""SELECT *
FROM `bigquery-public-data.ncaa_basketball.mbb_games_sr` """

# Set up the query
query_job = client.query(query)
# API request - run the query, and return a pandas DataFrame
gamesData = query_job.to_dataframe()
display(gamesData.head(5))

,game_id,season,status,coverage,neutral_site,scheduled_date,gametime,conference_game,tournament,tournament_type,...,a_fast_break_pts,a_second_chance_pts,a_team_turnovers,a_points_off_turnovers,a_team_rebounds,a_flagrant_fouls,a_player_tech_fouls,a_team_tech_fouls,a_coach_tech_fouls,created
0,b4451a02-26c5-4005-9ac8-b06c1f71e661,2015,closed,full,<NA>,2015-11-24,2015-11-24 21:30:00+00:00,<NA>,None,None,...,36,17,0,31,5,0,0,0,0,2018-02-20 15:48:58+00:00
1,b2f579ca-9eff-4b2b-a747-81169399c2e8,2015,closed,full,<NA>,2015-11-24,2015-11-24 02:00:00+00:00,<NA>,None,None,...,16,25,0,25,1,0,0,0,0,2018-02-20 15:48:53+00:00
2,571be71c-a5bf-446e-bf21-30eb6c54ac5e,2015,closed,full,<NA>,2015-11-25,2015-11-25 19:30:00+00:00,<NA>,None,None,...,6,6,0,12,0,0,0,0,0,2018-02-20 15:48:58+00:00
3,d6617923-0b23-49e4-af9b-9e4d0243e45c,2015,closed,full,<NA>,2015-12-19,2015-12-19 04:00:00+00:00,<NA>,None,None,...,2,13,1,15,3,0,0,0,0,2018-02-20 15:48:53+00:00
4,ffb463a4-dd3c-4ed9-b503-311b95ef0295,2015,closed,full,<NA>,2015-12-20,2015-12-20 04:00:00+00:00,<NA>,None,None,...,6,7,0,<NA>,0,0,0,0,0,2018-02-20 15:48:53+00:00


In [4]:
# Use this to create a csv of the data
# compression_opts = dict(method='zip',
#                         archive_name='ncaaBasketball.csv')  
# gamesData.to_csv('ncaaBasketball.zip', index=False,
#           compression=compression_opts)  

In [15]:
gamesData = pd.read_csv('ncaaBasketball/ncaaBasketball.csv')

postseason = gamesData[gamesData.tournament == 'NCAA']

In [45]:
preseason = gamesData[gamesData.tournament != 'NCAA']
dropMe=['tournament','status','coverage','tournament_type','tournament_round','tournament_game_no',
        'attendance','created','possession_arrow','venue_id','venue_city','venue_state','venue_address',
       'venue_zip','venue_country','venue_name','venue_capacity','h_league_id','h_league_name','h_league_alias',
        'h_conf_id','h_conf_name','h_conf_alias','h_division_id','h_division_name','h_division_alias',
        'h_logo_large','h_logo_medium','h_logo_small','a_league_id','a_league_name','a_league_alias',
        'a_conf_id','a_conf_name','a_conf_alias','a_division_id','a_division_name','a_division_alias',
        'a_logo_large','a_logo_medium','a_logo_small','h_name','a_name','h_market','a_market', 'h_id','a_id',
       'game_id','gametime','conference_game']

preseason = preseason.drop(columns = dropMe)

display(preseason.head(5))

,season,neutral_site,scheduled_date,lead_changes,times_tied,periods,h_alias,h_points_game,h_rank,h_minutes,...,a_points,a_fast_break_pts,a_second_chance_pts,a_team_turnovers,a_points_off_turnovers,a_team_rebounds,a_flagrant_fouls,a_player_tech_fouls,a_team_tech_fouls,a_coach_tech_fouls
0,2015,NaN,2015-11-24,3.0,2.0,2.0,CHA,73,0,3:20:00,...,93.0,36.0,17.0,0.0,31.0,5.0,0.0,0.0,0.0,0.0
1,2015,NaN,2015-11-24,1.0,0.0,2.0,CHA,72,0,3:20:00,...,123.0,16.0,25.0,0.0,25.0,1.0,0.0,0.0,0.0,0.0
2,2015,NaN,2015-11-25,2.0,0.0,2.0,CHA,93,0,3:20:00,...,100.0,6.0,6.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0
3,2015,NaN,2015-12-19,9.0,1.0,2.0,ORST,82,0,3:20:00,...,69.0,2.0,13.0,1.0,15.0,3.0,0.0,0.0,0.0,0.0
4,2015,NaN,2015-12-20,1.0,3.0,2.0,ORST,76,0,3:20:00,...,71.0,6.0,7.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0


In [89]:
# Make DataFrame to hold all the season statisics we're going to do
# preTeamStats = pd.DataFrame(preseason.h_alias.unique(),columns=['team'])
# preTeamStats = pd.DataFrame(preseason.groupby('h_alias'),columns=['pointsPerGame'])
preTeamStats = preseason.groupby(by = ['h_alias','season'])#.groupby('season').mean()


# preTeamStats = preseason.groupby('h_alias').mean()
# display(preseason.groupby('h_alias').head(5))
# preTeamStats[avgScore] = preseason[preseason.h_alias].h_points_game.mean()
display(preTeamStats.head())

,season,neutral_site,scheduled_date,lead_changes,times_tied,periods,h_alias,h_points_game,h_rank,h_minutes,...,a_points,a_fast_break_pts,a_second_chance_pts,a_team_turnovers,a_points_off_turnovers,a_team_rebounds,a_flagrant_fouls,a_player_tech_fouls,a_team_tech_fouls,a_coach_tech_fouls
0,2015,NaN,2015-11-24,3.0,2.0,2.0,CHA,73,0,3:20:00,...,93.0,36.0,17.0,0.0,31.0,5.0,0.0,0.0,0.0,0.0
1,2015,NaN,2015-11-24,1.0,0.0,2.0,CHA,72,0,3:20:00,...,123.0,16.0,25.0,0.0,25.0,1.0,0.0,0.0,0.0,0.0
2,2015,NaN,2015-11-25,2.0,0.0,2.0,CHA,93,0,3:20:00,...,100.0,6.0,6.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0
3,2015,NaN,2015-12-19,9.0,1.0,2.0,ORST,82,0,3:20:00,...,69.0,2.0,13.0,1.0,15.0,3.0,0.0,0.0,0.0,0.0
4,2015,NaN,2015-12-20,1.0,3.0,2.0,ORST,76,0,3:20:00,...,71.0,6.0,7.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29655,2014,NaN,2014-12-20,5.0,3.0,2.0,CHA,63,0,3:20:00,...,94.0,8.0,23.0,0.0,20.0,1.0,0.0,0.0,NaN,NaN
29658,2014,NaN,2014-11-18,8.0,6.0,2.0,HAW,54,0,3:20:00,...,62.0,0.0,12.0,0.0,11.0,8.0,0.0,0.0,NaN,NaN
29662,2014,NaN,2015-01-25,4.0,0.0,2.0,HAW,72,0,3:20:00,...,78.0,0.0,9.0,0.0,12.0,2.0,0.0,0.0,NaN,NaN
29693,2017,False,2018-02-11,4.0,4.0,2.0,HAW,71,0,3:20:00,...,77.0,2.0,9.0,0.0,19.0,2.0,0.0,1.0,NaN,NaN
